Study of the effect of national factors on home prices in the US
Task: Using publicly available data on the national factors that impact the supply and demand of homes in the US, build a data science model to study the effect of these variables on home prices.

Approach: The following variables are chosen for the study:

Unemployment Rate
Employment Rate
Per capita GDP
Median Household Income
Construction Prices
CPI
Interest Rates
The number of new houses supplied
Working Population
Urban Population
Percentage of population above 65
Housing subsidies
Number of Households
As a proxy for home prices, the S&P Case-Shiller Index is used.

Note: Most of the data is downloaded from [https://fred.stlouisfed.org/].

Data for all the variables is downloaded, preprocessed, and combined to create a dataset using the Extract Transform Load (ETL) method. Data for different variables had different frequencies. So, to combine the data, the necessary interpolations are made.

In [209]:
import numpy as np
import pandas as pd

In [210]:
# Reading CASE-SHILLER Index into a dataframe
df_CS = pd.read_csv("C:/Users/Vivek/Desktop/data/CSUSHPISA.csv")

# Changing dtype of date column
df_CS["DATE"] = pd.to_datetime(df_CS["DATE"])

# Selecting data till JULY 2023
mask = df_CS["DATE"] <= "2023-07-01"
df_CS = df_CS[mask]

#Resetting Index
df_CS.reset_index(inplace = True)
df_CS.drop(columns = ["index"], inplace = True)

# Creating "Year" and "Month" columns
df_CS["Year"] = pd.DatetimeIndex(df_CS["DATE"]).year
df_CS["Month"] = pd.DatetimeIndex(df_CS["DATE"]).month
print("Shape of the CASE-SHILLER Index:- ", df_CS.shape)
df_CS.tail()

Shape of the CASE-SHILLER Index:-  (259, 4)


,DATE,CSUSHPISA,Year,Month
254,2023-03-01,298.986,2023,3
255,2023-04-01,300.249,2023,4
256,2023-05-01,302.146,2023,5
257,2023-06-01,304.028,2023,6
258,2023-07-01,306.234,2023,7


In [211]:
df_unemp = pd.read_csv("C:/Users/Vivek/Desktop/data/UNRATE.csv")

print("Unemployment Rate Data:- ", df_unemp.shape)
df_unemp.tail()

Unemployment Rate Data:-  (259, 2)


,DATE,UNRATE
254,2023-03-01,3.5
255,2023-04-01,3.4
256,2023-05-01,3.6
257,2023-06-01,3.6
258,2023-07-01,3.5


In [212]:
df_emp = pd.read_csv("C:/Users/Vivek/Desktop/data/EMPRATE.csv")
df_emp = df_emp.rename(columns={'LREM64TTUSM156S': 'EmpRate'})

print("shape of the Employment Rate Data:- ", df_emp.shape)
df_emp.tail()

shape of the Employment Rate Data:-  (259, 2)


,DATE,EmpRate
254,2023-03-01,71.85808
255,2023-04-01,71.95604
256,2023-05-01,71.93774
257,2023-06-01,71.98534
258,2023-07-01,71.96878


In [213]:
# Reading Per Capita GDP Data into a dataframe
df_pcgdp = pd.read_csv("C:/Users/Vivek/Desktop/data/GDP_per_capita.csv")
print("Shape of the Per Capita GDP Data:- ", df_pcgdp.shape)
df_pcgdp.tail()

Shape of the Per Capita GDP Data:-  (87, 2)


,DATE,Per_Capita_GDP
82,2022-07-01,65939
83,2022-10-01,66341
84,2023-01-01,66673
85,2023-04-01,66341
86,2023-07-01,67039


In [214]:
df_Fed_rate = pd.read_csv("C:/Users/Vivek/Desktop/data/FEDFUNDS.csv")
print("Shape of the Interest rate data:- ",df_Fed_rate.shape)
df_Fed_rate.tail()

Shape of the Interest rate data:-  (259, 2)


,DATE,FEDFUNDS
254,2023-03-01,4.65
255,2023-04-01,4.83
256,2023-05-01,5.06
257,2023-06-01,5.08
258,2023-07-01,5.12


In [215]:
# Reading Construction Material Data into a dataframe
df_cons_price_index = pd.read_csv("C:/Users/Vivek/Desktop/data/construction_price_ppi.csv")
print("Shape of the Construction Material Data:- ", df_cons_price_index.shape)
df_cons_price_index.tail()

Shape of the Construction Material Data:-  (259, 2)


,DATE,Cons_Material
254,2023-03-01,331.729
255,2023-04-01,333.366
256,2023-05-01,337.473
257,2023-06-01,337.336
258,2023-07-01,334.512


In [216]:
# Consumer Price Index
df_CPI = pd.read_csv("C:/Users/Vivek/Desktop/data/CPIAUCSL.csv")
print("Shape of the Consumer Price Index:- ", df_CPI.shape)
df_CPI.tail()

Shape of the Consumer Price Index:-  (259, 2)


,DATE,CPI
254,2023-03-01,301.643
255,2023-04-01,302.858
256,2023-05-01,303.316
257,2023-06-01,304.099
258,2023-07-01,304.615


In [217]:
# Monthly new house supply
df_house = pd.read_csv("C:/Users/Vivek/Desktop/data/MSACSR.csv")

print("Shape of the monthly house supply data:- ", df_house.shape)
df_house.tail()

Shape of the monthly house supply data:-  (259, 2)


,DATE,Houses
254,2023-03-01,8.1
255,2023-04-01,7.5
256,2023-05-01,6.9
257,2023-06-01,7.7
258,2023-07-01,7.3


In [218]:
df_oldpop = pd.read_csv("C:/Users/Vivek/Desktop/data/old_age_pop.csv")

print("Shape of the population data age above 65:- ", df_oldpop.shape)
df_oldpop.tail()

Shape of the population data age above 65:-  (21, 2)


,DATE,old_age_pop
16,2018-01-01,15.4
17,2019-01-01,15.8
18,2020-01-01,16.2
19,2021-01-01,16.7
20,2022-01-01,17.1


In [219]:
# Urban Population Percent
df_urban = pd.read_csv("C:/Users/Vivek/Desktop/data/urban_pop.csv")

print("Shape of the urban population percent data:- ", df_urban.shape)
df_urban.tail()

Shape of the urban population percent data:-  (21, 2)


,DATE,urban_pop_us
16,2018-01-01,82.3
17,2019-01-01,82.5
18,2020-01-01,82.7
19,2021-01-01,82.9
20,2022-01-01,83.1


In [220]:
# Housing Subsidies

df_subsidy = pd.read_csv("C:/Users/Vivek/Desktop/data/housing_subsidies.csv")

print("Shape of the housing subsidies:- ", df_subsidy.shape)
df_subsidy.tail()

Shape of the housing subsidies:-  (21, 2)


,DATE,Subsidy
16,2018-01-01,38.859
17,2019-01-01,40.185
18,2020-01-01,44.147
19,2021-01-01,45.299
20,2022-01-01,48.021


In [221]:
# Working age population

df_working = pd.read_csv("C:/Users/Vivek/Desktop/data/working_age_population.csv")

print("Shape of the working age population:- ", df_working.shape)
df_working.tail()

Shape of the working age population:-  (259, 2)


,DATE,working_age_pop
254,2023-03-01,208145900
255,2023-04-01,208264600
256,2023-05-01,208483500
257,2023-06-01,208656700
258,2023-07-01,208816400


In [222]:
# Real Median Household Income

df_income = pd.read_csv("C:/Users/Vivek/Desktop/data/median_household_income.csv")

print("Shape of the median household income data:- ", df_income.shape)
df_income.tail()

Shape of the median household income data:-  (21, 2)


,DATE,median_income
16,2018-01-01,75790
17,2019-01-01,81210
18,2020-01-01,79560
19,2021-01-01,79260
20,2022-01-01,77540


In [223]:
# Total number of households

df_households = pd.read_csv("C:/Users/Vivek/Desktop/data/household.csv")

print("Shape of the total households data:- ", df_households.shape)
df_households.tail()

Shape of the total households data:-  (21, 2)


,DATE,Num_Households
16,2018-01-01,127586
17,2019-01-01,128579
18,2020-01-01,128451
19,2021-01-01,129224
20,2022-01-01,131202


In [224]:
# Merging Per Capita GDP (Quarterly data)
df_pcgdp["DATE"] = pd.to_datetime(df_pcgdp["DATE"])
df_CS = pd.merge(df_CS,df_pcgdp, how = "left")
df_CS.head()

,DATE,CSUSHPISA,Year,Month,Per_Capita_GDP
0,2002-01-01,117.144,2002,1,50091.0
1,2002-02-01,117.844,2002,2,NaN
2,2002-03-01,118.687,2002,3,NaN
3,2002-04-01,119.611,2002,4,50286.0
4,2002-05-01,120.724,2002,5,NaN


In [225]:
# Concating dataframes having monthly data to create one dataframe
df = pd.DataFrame()
df_bymonth = [df_CS, df_working, df_house, df_CPI, df_unemp, df_emp, df_cons_price_index, df_Fed_rate]
for df1 in df_bymonth:
    df1["DATE"] = pd.to_datetime(df1["DATE"])
    df1 = df1.set_index("DATE")
    df = pd.concat([df,df1], axis = 1)
print(df.shape)
df.head()

(259, 11)


,CSUSHPISA,Year,Month,Per_Capita_GDP,working_age_pop,Houses,CPI,UNRATE,EmpRate,Cons_Material,FEDFUNDS
DATE,,,,,,,,,,,
2002-01-01,117.144,2002,1,50091.0,182668900,4.2,177.7,5.7,72.03223,142.0,1.73
2002-02-01,117.844,2002,2,NaN,182824700,4.0,178.0,5.7,72.33686,142.2,1.74
2002-03-01,118.687,2002,3,NaN,183083800,4.1,178.5,5.7,72.15416,143.2,1.73
2002-04-01,119.611,2002,4,50286.0,183315900,4.3,179.3,5.9,71.90136,143.5,1.75
2002-05-01,120.724,2002,5,NaN,183461200,4.0,179.5,5.8,72.01969,143.8,1.75


In [226]:
# Merging other dataframes 
others = [df_urban, df_households, df_income, df_subsidy, df_oldpop]
for df1 in others:
    if "Year" not in df1.columns:
        df1["Year"] = pd.DatetimeIndex(df1["DATE"]).year
        df1.set_index("DATE", inplace = True)
        df = pd.merge(df, df1, how = "left", on = "Year")
    else:
        df1.set_index("DATE", inplace = True)
        df = pd.merge(df, df1, how = "left", on = "Year")
df["DATE"] = df_CS["DATE"]
df.set_index("DATE", inplace = True)
df.head()

,CSUSHPISA,Year,Month,Per_Capita_GDP,working_age_pop,Houses,CPI,UNRATE,EmpRate,Cons_Material,FEDFUNDS,urban_pop_us,Num_Households,median_income,Subsidy,old_age_pop
DATE,,,,,,,,,,,,,,,,
2002-01-01,117.144,2002,1,50091.0,182668900,4.2,177.7,5.7,72.03223,142.0,1.73,79.1,109297.0,68310.0,24.183,12.3
2002-02-01,117.844,2002,2,NaN,182824700,4.0,178.0,5.7,72.33686,142.2,1.74,79.1,109297.0,68310.0,24.183,12.3
2002-03-01,118.687,2002,3,NaN,183083800,4.1,178.5,5.7,72.15416,143.2,1.73,79.1,109297.0,68310.0,24.183,12.3
2002-04-01,119.611,2002,4,50286.0,183315900,4.3,179.3,5.9,71.90136,143.5,1.75,79.1,109297.0,68310.0,24.183,12.3
2002-05-01,120.724,2002,5,NaN,183461200,4.0,179.5,5.8,72.01969,143.8,1.75,79.1,109297.0,68310.0,24.183,12.3


In [227]:
print(df.shape)

(259, 16)


In [228]:
df.isna().sum()

CSUSHPISA            0
Year                 0
Month                0
Per_Capita_GDP     172
working_age_pop      0
Houses               0
CPI                  0
UNRATE               0
EmpRate              0
Cons_Material        0
FEDFUNDS             0
urban_pop_us         7
Num_Households       7
median_income        7
Subsidy              7
old_age_pop          7
dtype: int64

In [229]:
# Filling missing values in the Per_Capita_GDP column using linear interpolation
df["Per_Capita_GDP"] = df["Per_Capita_GDP"].interpolate()

In [230]:
df

,CSUSHPISA,Year,Month,Per_Capita_GDP,working_age_pop,Houses,CPI,UNRATE,EmpRate,Cons_Material,FEDFUNDS,urban_pop_us,Num_Households,median_income,Subsidy,old_age_pop
DATE,,,,,,,,,,,,,,,,
2002-01-01,117.144,2002,1,50091.000000,182668900,4.2,177.700,5.7,72.03223,142.000,1.73,79.1,109297.0,68310.0,24.183,12.3
2002-02-01,117.844,2002,2,50156.000000,182824700,4.0,178.000,5.7,72.33686,142.200,1.74,79.1,109297.0,68310.0,24.183,12.3
2002-03-01,118.687,2002,3,50221.000000,183083800,4.1,178.500,5.7,72.15416,143.200,1.73,79.1,109297.0,68310.0,24.183,12.3
2002-04-01,119.611,2002,4,50286.000000,183315900,4.3,179.300,5.9,71.90136,143.500,1.75,79.1,109297.0,68310.0,24.183,12.3
2002-05-01,120.724,2002,5,50311.333333,183461200,4.0,179.500,5.8,72.01969,143.800,1.75,79.1,109297.0,68310.0,24.183,12.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-01,298.986,2023,3,66451.666667,208145900,8.1,301.643,3.5,71.85808,331.729,4.65,NaN,NaN,NaN,NaN,NaN
2023-04-01,300.249,2023,4,66341.000000,208264600,7.5,302.858,3.4,71.95604,333.366,4.83,NaN,NaN,NaN,NaN,NaN
2023-05-01,302.146,2023,5,66573.666667,208483500,6.9,303.316,3.6,71.93774,337.473,5.06,NaN,NaN,NaN,NaN,NaN


In [231]:
df.dropna(inplace = True)

In [232]:
df.isna().sum()

CSUSHPISA          0
Year               0
Month              0
Per_Capita_GDP     0
working_age_pop    0
Houses             0
CPI                0
UNRATE             0
EmpRate            0
Cons_Material      0
FEDFUNDS           0
urban_pop_us       0
Num_Households     0
median_income      0
Subsidy            0
old_age_pop        0
dtype: int64

In [233]:
print("Shape of the dataframe after preprocessing:- ", df.shape)

Shape of the dataframe after preprocessing:-  (252, 16)


In [234]:
df.to_csv("prepared_dataset.csv")

In [235]:
us_house_price_df = pd.read_csv("prepared_dataset.csv").set_index("DATE")
us_house_price_df.head()

,CSUSHPISA,Year,Month,Per_Capita_GDP,working_age_pop,Houses,CPI,UNRATE,EmpRate,Cons_Material,FEDFUNDS,urban_pop_us,Num_Households,median_income,Subsidy,old_age_pop
DATE,,,,,,,,,,,,,,,,
2002-01-01,117.144,2002,1,50091.000000,182668900,4.2,177.7,5.7,72.03223,142.0,1.73,79.1,109297.0,68310.0,24.183,12.3
2002-02-01,117.844,2002,2,50156.000000,182824700,4.0,178.0,5.7,72.33686,142.2,1.74,79.1,109297.0,68310.0,24.183,12.3
2002-03-01,118.687,2002,3,50221.000000,183083800,4.1,178.5,5.7,72.15416,143.2,1.73,79.1,109297.0,68310.0,24.183,12.3
2002-04-01,119.611,2002,4,50286.000000,183315900,4.3,179.3,5.9,71.90136,143.5,1.75,79.1,109297.0,68310.0,24.183,12.3
2002-05-01,120.724,2002,5,50311.333333,183461200,4.0,179.5,5.8,72.01969,143.8,1.75,79.1,109297.0,68310.0,24.183,12.3


Step 1 completed here,